# Home task: pandas

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
#Можна це закоментувати і отримати числа в науковому форматі
pd.set_option('display.float_format', '{:.2f}'.format)

#Зчитую данні про енергетику країн пропускаючи непотрібну інформацію.
energy = pd.read_excel('Energy Indicators.xls',skiprows=17, usecols='C:F', nrows=227, names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])

#Забираю ... , заміняю на Nan
energy.replace("...", np.nan, inplace=True)

#Перетворюю петаджоулі в гігаджоулі
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

#Перейменовую деякі країни
energy['Country'].replace(
    ['Republic of Korea',
     'United States of America20',
     'United Kingdom of Great Britain and Northern Ireland19',
     'China, Hong Kong Special Administrative Region3'], 
     
    ['South Korea',
     'United States',
     'United Kingdom',
     'Hong Kong'], 
     inplace=True)

#Прибираю в країнах числа
energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)
#Прибираю в країнах дужки
energy['Country'] = energy['Country'].str.replace(r'\s\(.+', '', regex=True)

#Зчитую ВВП країн
GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_5871885.csv')

#Перейменовую деякі країни
GDP['Country Name'].replace(
    ['Korea, Rep.',
     'Iran, Islamic Rep.',
     'Hong Kong SAR, China'], 
     
    ['South Korea',
     'Iran',
     'Hong Kong'], 
     inplace=True)

#Залишаю лише проміжок часу між 2006 - 2015
GDP = GDP[['Country Name'] + [str(year) for year in range(2006, 2016)]]

#Зчитую внески в документацію від кожної країни
ScimEn = pd.read_excel('scimagojr country rank 1996-2022.xlsx')

#Залишаю, лише топ 15 Rank
ScimEn = ScimEn[ScimEn['Rank'] <= 15]

#Забираю колонку Region
ScimEn.drop('Region', axis=1, inplace=True)


def answer_one():
    #Робимо merge по колонці Country
    first_merge = pd.merge(ScimEn, energy, how='inner', on='Country')
    
    #Робимо фінальний merge по колонці Country з колнкою Country Name
    df = pd.merge(first_merge, GDP, how='inner', left_on='Country', right_on='Country Name')

    #Забираємо лишню колонку Country Name
    df.drop('Country Name', axis=1, inplace=True)

    #Робимо колонку Country індексною
    df.set_index('Country', inplace=True)

    return df

print(answer_one())

                    Rank  Documents  Citable documents  Citations  Self-citations  Citations per document  H index   Energy Supply  Energy Supply per Capita  % Renewable              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015
Country                                                                                                                                                                                                                                                                                                                                                      
China                  1     360468             358777    3947871         2705774                   10.95      308 127191000000.00                     93.00        19.75  2752118542130.40  3550327569655.47  4594336785737.67  5101691085625.95  6087191720866.69  7551545321799.88  8532185615746.62  957

# Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named avgGDP with 15 countries and their average GDP sorted in descending order.*

In [21]:
def answer_two():
    Top15 = answer_one()
    avgGDP = Top15.loc[:,'2006':'2015'].mean(axis=1)
    avgGDP = avgGDP.sort_values(ascending=False)
    return avgGDP

print(answer_two())

Country
United States        15704029340300.00
China                 6927706520148.28
Japan                 5239642077099.60
Germany               3523341675097.46
United Kingdom        2780276197282.68
France                2691336579143.64
Italy                 2142986168498.18
Brazil                1988889078193.37
Russian Federation    1666745628516.50
Canada                1616359264592.76
India                 1602352072666.71
Spain                 1400886445421.01
South Korea           1221371625499.97
Australia             1207512888804.88
Iran                   456326122839.82
dtype: float64


# Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [22]:
def answer_three():
    Top15 = answer_one()
    startGDP = Top15.loc['France','2006']
    currentGDP = Top15.loc['France','2015']
    return currentGDP - startGDP
print(answer_three())

118652421857.7998


# Question 4
Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [24]:
def answer_four():
    Top15 = answer_one()
    Top15['ratio'] = Top15['Self-citations'] / Top15['Citations']

    max_ratio_country = Top15['ratio'].idxmax()
    max_ratio_value = Top15['ratio'].max()
    

    return (max_ratio_country, max_ratio_value)
print(answer_four())

('China', 0.6853754846599598)


# Question 5
Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [34]:
def answer_five():
    Top15 = answer_one()
    Top15['population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['population'].sort_values(ascending=False).index[2]
print(answer_five())

United States


# Question 6
Create a column that estimates the number of citable documents per person. What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).

*This function should return a single number.*

In [37]:
def answer_six():
    Top15 = answer_one()
    Top15['population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable documents per person'] = Top15['Citable documents'] / Top15['population']
    correlation = Top15['Citable documents per person'].corr(Top15['Energy Supply per Capita'])

    return correlation
print(answer_six())

0.7114342527331208


# Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

*This function should return a DataFrame with index named Continent ['Asia', 'Australia', 'Europe', 'North America', 'South America'] and columns ['size', 'sum', 'mean', 'std']*

In [12]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_seven():
    Top15 = answer_one()

    Top15['population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']

    Top15['Continent'] = Top15.index.to_series().map(ContinentDict)

    result = Top15.groupby('Continent')['population'].agg(['size', 'sum', 'mean', 'std'])
    return result

print(answer_seven())

               size           sum         mean          std
Continent                                                  
Asia              5 2898666386.61 579733277.32 679097888.37
Australia         1   23316017.32  23316017.32          NaN
Europe            6  457929667.22  76321611.20  34647667.07
North America     2  352855249.48 176427624.74 199669644.86
South America     1  205915254.24 205915254.24          NaN
